In [0]:
!unzip data.zip

Archive:  data.zip
  inflating: data.pickle             
  inflating: label.pickle            


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "drive/My Drive/traffic_model/data.pickle" .
!cp "drive/My Drive/traffic_model/label.pickle" .

In [2]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf

x = pickle.load(open('data.pickle','rb'))
y = pickle.load(open('label.pickle','rb'))

In [3]:
print(np.shape(x))
x=np.reshape(x,(508,299,299,3))
print(np.shape(x))

(508, 1, 299, 299, 3)
(508, 299, 299, 3)


In [4]:
print(np.shape(y))
y = np.asarray(y)
print(y)

(508, 2)
[[0 0]
 [1 0]
 [1 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [0]:
y1 = np.empty((508,), dtype = int)
y2 = np.empty((508,), dtype = int)

y1[:] = y[:,0]
y2[:] = y[:,1]

In [6]:
from tensorflow.keras import regularizers, Model
from tensorflow.keras.metrics import Recall,Precision
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Activation,Conv2D,MaxPooling2D,Dropout,Input
import tensorflow as tf


model0 = Input(shape = (299,299,3))

model = Conv2D(64, (3,3), activation='relu')(model0)
model = MaxPooling2D(2, 2)(model)

model = Conv2D(128, (3,3), activation='relu')(model)
model = MaxPooling2D(2, 2)(model)

model = Conv2D(128, (3,3), activation='relu')(model)
model = MaxPooling2D(2, 2)(model)

model = Conv2D(128, (3,3), activation='relu')(model)
model = MaxPooling2D(2, 2)(model)

model = Flatten()(model)

model = Dense(128,kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.01), activation = 'relu')(model)

model = Dense(64,kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.01), activation = 'relu')(model)

model = Dropout(0.2)(model)

model = Dense(32,kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.01), activation = 'relu')(model)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
model1 = Dense(1, activation = "sigmoid")(model)

model2 = Dense(1, activation = "sigmoid")(model)

final = Model(inputs = [model0], outputs = [model1, model2])

final.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

final.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 297, 297, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 148, 148, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 146, 146, 128 73856       max_pooling2d[0][0]              


In [8]:
import keras
from tensorflow.keras.callbacks import EarlyStopping,CSVLogger,ModelCheckpoint,TensorBoard
filename="metrics.csv"
es=EarlyStopping(monitor='loss', min_delta=0, patience=3, mode='min', baseline=None, restore_best_weights=True)
csv=CSVLogger(filename, separator=',', append=False)
checkpoint=ModelCheckpoint("weights.{epoch}-{val_acc:.2f}.hdf5", monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
tensorboard=TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

Using TensorFlow backend.


In [9]:
final.fit(x, [y1,y2], epochs=25, batch_size=64, callbacks=[tensorboard])

Train on 508 samples
Epoch 1/25
508/508 [==============================] - 14s 29ms/sample - loss: 16.0400 - dense_3_loss: 1.9794 - dense_4_loss: 2.2003 - dense_3_acc: 0.5394 - dense_4_acc: 0.4843
Epoch 2/25
508/508 [==============================] - 5s 10ms/sample - loss: 4.8416 - dense_3_loss: 0.6878 - dense_4_loss: 0.6866 - dense_3_acc: 0.7598 - dense_4_acc: 0.7618
Epoch 3/25
508/508 [==============================] - 5s 10ms/sample - loss: 4.7855 - dense_3_loss: 0.6826 - dense_4_loss: 0.6808 - dense_3_acc: 0.7598 - dense_4_acc: 0.7638
Epoch 4/25
508/508 [==============================] - 5s 10ms/sample - loss: 4.6418 - dense_3_loss: 0.6758 - dense_4_loss: 0.6744 - dense_3_acc: 0.7598 - dense_4_acc: 0.7657
Epoch 5/25
508/508 [==============================] - 5s 10ms/sample - loss: 4.4679 - dense_3_loss: 0.6696 - dense_4_loss: 0.6685 - dense_3_acc: 0.7598 - dense_4_acc: 0.7657
Epoch 6/25
508/508 [==============================] - 5s 10ms/sample - loss: 4.2978 - dense_3_loss: 0.6630 

In [10]:
final.evaluate(x, [y1,y2], verbose=1)

508/508 [==============================] - 4s 7ms/sample - loss: 2.6282 - dense_3_loss: 0.5698 - dense_4_loss: 0.5594 - dense_3_acc: 0.7598 - dense_4_acc: 0.7657


[2.628245781725786, 0.5697989, 0.55943877, 0.7598425, 0.765748]

In [0]:
final.save('model_1.h5')

In [12]:
!pwd

/content


In [0]:
import zipfile
import os
def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

In [0]:
zipfolder('logs',"./logs")

In [0]:
!rm -rf logs